In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import explode,col,expr
import pandas as pd


In [2]:
spark = SparkSession.builder.appName('task_3').getOrCreate()

25/04/28 17:49:25 WARN Utils: Your hostname, nischit-baral resolves to a loopback address: 127.0.1.1; using 10.10.42.113 instead (on interface enp2s0)
25/04/28 17:49:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 17:49:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df_pyspark = spark.read.option('multiline','true').json('works.json')


In [5]:
# df_pyspark.show()
df_pyspark.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- provider_groups: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- npi: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- tin: struct (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)



In [6]:
provider_group = df_pyspark.withColumn("new_provider", explode("provider_groups"))

provider_again = provider_group.select(
    "provider_group_id",
    col("new_provider.npi").alias("npi"),
    col("new_provider.tin.type").alias("tin_type"),
    col("new_provider.tin.value").alias("tin_value")
)         

provider_again.show()

+-----------------+--------------------+--------+----------+
|provider_group_id|                 npi|tin_type| tin_value|
+-----------------+--------------------+--------+----------+
|         10001001|[1235233008, 1316...|     ein|04-3267217|
|         10001001|        [1366459570]|     ein|11-1562701|
|         10001001|        [1417915653]|     ein|11-3358535|
|         10001001|        [1417915653]|     ein|13-3888838|
|         10002001|[1609829761, 1821...|     ein|00-0004110|
|         10002001|        [1760986277]|     ein|00-6980743|
|         10002001|        [1215075882]|     ein|01-0550744|
|         10002001|        [1013917665]|     ein|01-0555304|
|         10002001|[1679780811, 1700...|     ein|01-0555483|
|         10002001|        [1376647511]|     ein|01-0567880|
|         10002001|        [1740287010]|     ein|01-0569673|
|         10002001|        [1265538219]|     ein|01-0571165|
|         10002001|        [1689625253]|     ein|01-0577025|
|         10002001|     

In [7]:
rem_hyp = provider_again.withColumn('tin_value', expr("replace(tin_value,'-','')"))
rem_hyp.show()

+-----------------+--------------------+--------+---------+
|provider_group_id|                 npi|tin_type|tin_value|
+-----------------+--------------------+--------+---------+
|         10001001|[1235233008, 1316...|     ein|043267217|
|         10001001|        [1366459570]|     ein|111562701|
|         10001001|        [1417915653]|     ein|113358535|
|         10001001|        [1417915653]|     ein|133888838|
|         10002001|[1609829761, 1821...|     ein|000004110|
|         10002001|        [1760986277]|     ein|006980743|
|         10002001|        [1215075882]|     ein|010550744|
|         10002001|        [1013917665]|     ein|010555304|
|         10002001|[1679780811, 1700...|     ein|010555483|
|         10002001|        [1376647511]|     ein|010567880|
|         10002001|        [1740287010]|     ein|010569673|
|         10002001|        [1265538219]|     ein|010571165|
|         10002001|        [1689625253]|     ein|010577025|
|         10002001|        [1184901563]|

In [13]:
rem_hyp.toPandas().to_json('in_prov.json', orient='records')